# Modelamiento
----------------------------------------------

MLDS 6 - Grupo 8


### Librerias y configuraciones

In [54]:
import numpy as np
import pandas as pd
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [52]:
def list_confusion_matrix(cm,classes):
  df = pd.DataFrame(data = cm,
                    index = pd.MultiIndex.from_product([['Valor real'], classes]),
                    columns = pd.MultiIndex.from_product([['Valor predicho'], classes]))

  return df

## Lectura de sub conjunto de datos de train y test
-----------------------------------

Se cargan los datos resultado de la sección de preprocesamiento, y se verifica que no haya un cambio en sus dimensiones.

In [6]:
data = np.load("/home/kevo/Escritorio/mlds6/tdsp_template_grupo8/scripts/preprocessing/train_test_array.npz")
X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test']
y_test = data['y_test']
print(f'Dimensiones X_train: {X_train.shape}')
print(f'Dimensiones X_test: {X_test.shape}')
print(f'Dimensiones y_train: {y_train.shape}')
print(f'Dimensiones y_test: {y_test.shape}')

Dimensiones X_train: (76916, 15)
Dimensiones X_test: (19230, 15)
Dimensiones y_train: (76916,)
Dimensiones y_test: (19230,)


## Análisis de modelos de Machine learning
---------------------------------------
En esta etapa se utilizaran algoritmos supervisados con validacion cruzada K-fold para encontrar el mejor modelo y los hiperparametros mas adecuados para la clasificación que se desea obtener.

## Regresion logistica
----------------------------------------
Se selecciono la técnica de regresión logística debido a su eficiencia y robustez para clasificación binaria, ademas de su eficiecnia computacional. Para la validación cruzada se tendran en cuenta, hiperparametros como: el tipo de optimizacion, solver, y las iteraciones maximas. 

In [21]:
param_grid = {
    'penalty': ['l2', None],
    'solver': ['lbfgs'],
    'max_iter': [100, 200, 400, 800],
}

Se utiliza la herramienta gridsearchcv para la que se configura para 5 particiones al realizar CV

In [43]:
grid_lg = GridSearchCV(LogisticRegression(random_state=3,),
                   param_grid=param_grid,
                   verbose=1,
                   return_train_score=True,
                   cv = 5
                   )
grid_lg.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5, estimator=LogisticRegression(random_state=3),
             param_grid={'max_iter': [100, 200, 400, 800],
                         'penalty': ['l2', None], 'solver': ['lbfgs']},
             return_train_score=True, verbose=1)

Se realiza un dataframe que contiene los mejores score de los primeros modelos entrenados, este indica los hiperparametros seleccionados en cada iteración, asi como las metricas obtenicas por cada split y su respectivo promedio y desviacion estandar al final.

In [44]:
pd.DataFrame(grid_lg.cv_results_).sort_values(by='rank_test_score', ascending=True).head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_iter,param_penalty,param_solver,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.315356,0.029572,0.006032,0.000161,100,l2,lbfgs,"{'max_iter': 100, 'penalty': 'l2', 'solver': '...",0.958788,0.958591,...,0.958734,0.001239,1,0.958656,0.958884,0.958543,0.958364,0.959258,0.958741,0.000309
2,0.285413,0.053993,0.005806,0.000208,200,l2,lbfgs,"{'max_iter': 200, 'penalty': 'l2', 'solver': '...",0.958788,0.958591,...,0.958734,0.001239,1,0.958656,0.958884,0.958543,0.958364,0.959258,0.958741,0.000309
6,0.555815,0.157849,0.019346,0.016326,800,l2,lbfgs,"{'max_iter': 800, 'penalty': 'l2', 'solver': '...",0.958788,0.958591,...,0.958734,0.001239,1,0.958656,0.958884,0.958543,0.958364,0.959258,0.958741,0.000309


Se extrae los mejores parametros del modelo, que en el caso de la regresión logística es, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs', logrando un score de 0.958 para entrenamiento y 0.9596 con datos de prueba. Lo que indica que la variacion de accuraccy utilizando datos no vistos en el entrenamiento y datos si utilizados es muy pequeña, sin embargo para saber si es un modelo bueno se necesita conocer otras metricas.

In [60]:
print(grid_lg.best_params_)
print(grid_lg.best_score_)
print(grid_lg.score(X_test, y_test))

{'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.9587342028427285
0.959594383775351


Se almacena el mejor modelo de LR con joblib, para evitar volver a aplicar gridsearchCV

In [61]:
lg_bm = grid_lg.best_estimator_
joblib.dump(lg_bm, 'lg_best_model.joblib')

['lg_best_model.joblib']

Se carga el mejor modelo y se reliza una predicción

In [62]:
model_lg = joblib.load("lg_best_model.joblib")
y_pred_lg = model_lg.predict(X_test)

Con las prediccion se genera un clasification report para saber como esta el rendimiento de LR.

In [63]:
print(classification_report(y_test,y_pred_lg, target_names=['No_Diabetes', 'Diabetes']))

              precision    recall  f1-score   support

 No_Diabetes       0.97      0.99      0.98     17534
    Diabetes       0.87      0.64      0.73      1696

    accuracy                           0.96     19230
   macro avg       0.92      0.81      0.86     19230
weighted avg       0.96      0.96      0.96     19230



Se genera un matriz de confusión que indica que la mayoria de casos con personas que no padecen diabetes es clasificado satisfactoriamente, pero con respecto a las personas que padecen diabetes, un porcentaje considerable esta mal clasificado lo que indica que este algoritmo tiene problemas para clasificar algunos casos en los cuales la diabetes es positiva. 

In [64]:
lg_mat = confusion_matrix(y_test, y_pred_lg)
list_confusion_matrix(lg_mat, ['No_Diabetes', 'Diabetes'])

Valor predicho         
                          No_Diabetes Diabetes
Valor real No_Diabetes          17376      158
           Diabetes               619     1077

# C-Support Vector Classification.
--------------------------------------------
Otro algorimo utilizado es el super vector machine para clasificación, que se caraxteriza por buscar una region optima que separe diferentes clases de datos en un margen considerable. Es decir que buscar un hiperplano que maxime la distancia entre las clases que se necesite diferenciar, ademas que puede trabajar en casos de linealidad y no linealidad. Para nuestro caso vamos a probar, variando hiperparametros como el tipo de kernel, la tasa de optimizacion "C" y el valor de gamma.

In [32]:
param_grid_svc = {
    'kernel': ['linear','rbf', 'sigmoid'],
    'C': [0.001 ,0.01, 0.1],
    'gamma': ['scale', 'auto', float]
}

Se genera la herramiento gridSearchCv, configruada para 5 folds

In [33]:
grid_svc = GridSearchCV(SVC(random_state=3),
                   param_grid=param_grid_svc,
                   verbose=1,
                   return_train_score=True,
                   cv = 5
                   )
grid_svc.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(random_state=3),
             param_grid={'C': [0.001, 0.01, 0.1],
                         'gamma': ['scale', 'auto', <class 'float'>],
                         'kernel': ['linear', 'rbf', 'sigmoid']},
             return_train_score=True, verbose=1)

Se crea un dataframe con los resultados de los 3 primeros modelos, donde encuentra los hiperparateros, y las metricas especificas y generales de cada iteración.

In [36]:
pd.DataFrame(grid_svc.cv_results_).sort_values(by='rank_test_score', ascending=True).head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
21,18.610979,0.091407,3.214798,0.013363,0.1,auto,linear,"{'C': 0.1, 'gamma': 'auto', 'kernel': 'linear'}",0.959958,0.958526,...,0.958526,0.001095,1,0.958282,0.958738,0.958429,0.958266,0.959144,0.958572,0.000332
18,18.761773,0.088537,3.216793,0.022473,0.1,scale,linear,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}",0.959958,0.958526,...,0.958526,0.001095,1,0.958282,0.958738,0.958429,0.958266,0.959144,0.958572,0.000332
19,29.470915,0.232364,9.436970,0.034859,0.1,scale,rbf,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}",0.958398,0.956835,...,0.957083,0.001090,3,0.957161,0.957437,0.957177,0.957161,0.957974,0.957382,0.000314


Por separado, se extraen del mejor modelo, sus hiperametros y sus metricas de entrenamiento y prueba.

In [65]:
print(grid_svc.best_estimator_)
print(grid_svc.best_score_)
print(grid_svc.score(X_test, y_test))

SVC(C=0.1, kernel='linear', random_state=3)
0.9585261657882294
0.9593863754550181


Se almacena el mejor modelo de SVC para evitar volver a aplicar la CV

In [66]:
svc_bm = grid_svc.best_estimator_
joblib.dump(svc_bm, 'svc_best_model.joblib')

['svc_best_model.joblib']

Se carga el mejor modelo y se obtienen predicciones

In [67]:
model_svc = joblib.load("svc_best_model.joblib")
y_pred_svc = model_svc.predict(X_test)

Se obtiene el clasification report de SVC

In [68]:
print(classification_report(y_test,y_pred_svc, target_names=['No_Diabetes', 'Diabetes']))

              precision    recall  f1-score   support

 No_Diabetes       0.96      1.00      0.98     17534
    Diabetes       0.94      0.58      0.72      1696

    accuracy                           0.96     19230
   macro avg       0.95      0.79      0.85     19230
weighted avg       0.96      0.96      0.95     19230



La matriz de confusión muestra que solo 78 casos de no diabetes fueron mal clasificados por lo que el algrotimo funciona bien clasificando este tipo. Pero, el porcentaje de casos de diabetes que fueron bien clasificados estan cercanos a los que no fueron bien clasificados, lo que indica que el algoritmo tiene problemas al decidir a que tipo de clase pertenece un registro u otro de manera mas frecuente.

In [69]:
svc_mat = confusion_matrix(y_test, y_pred_svc)
list_confusion_matrix(svc_mat, ['No_Diabetes', 'Diabetes'])

Valor predicho         
                          No_Diabetes Diabetes
Valor real No_Diabetes          17466       68
           Diabetes               713      983